In [1]:
import spacy
import pandas as pd
import numpy as np
import neuralcoref
import ast
import random
from tqdm import tqdm_notebook
import urllib.request
from PIL import Image


scoring = pd.read_csv('scoring.csv', delimiter = ";")
dict_val = {}

for i in range(48):
    dict_val[scoring['function'][i]] = scoring['score_norm'][i]

nlp=spacy.load("en_core_web_md")
neuralcoref.add_to_pipe(nlp,greedyness=0.5)

C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.morphology.Morphology size changed, may indicate binary incompatibility. Expected 104 from C header, got 112 from PyObject
  return f(*args, **kwds)
C:\Users\THINKPAD\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: spacy.vocab.Vocab size changed, may indicate binary incompatibility. Expected 96 from C header, got 104 from PyObject
  return f(*args, **kwds)
C:\Users\THINKPAD\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (10,15,17,23,24,29,30,37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
def author_del_photos(text):
    text_s = text
    while(text_s[-7:] != ' / AFP ' and len(text_s) > 1): 
        text_s = text_s[:-1]
    if(len(text_s) <= 1):
        return(text)
    return(text_s[:-7])

In [3]:
def transfo_event(ev):
    res=ev
    return res[1:-1].replace("'","").split(', ')

df_depeche=pd.read_csv('depeches_clean.csv',index_col='Unnamed: 0')
df_depeche.event=df_depeche.event.apply(transfo_event)

df2=pd.read_csv('photos_clean.csv',index_col='Unnamed: 0')
df2.event=df2.event.apply(transfo_event)

df2.caption = df2.caption.apply(author_del_photos)

C:\Users\THINKPAD\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (11,16,24,25,30,31,38) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [15]:
def read_scores(csv):
    file=pd.read_csv(csv)
    res=[]
    for string in file.scores:
        res.append(ast.literal_eval(string))
    return res


liste_score2=read_scores("event_score_2.csv")
liste_score4=read_scores("event_score_4.csv")

liste_score2_d=read_scores("depeche_score_2.csv")
liste_score4_d=read_scores("depeche_score_4.csv")

Similarité en normalisant par le max.  Pour best5_image_test, j correspond a l'indice de l'image dont on veut tester si elle est dans les 5 meilleures images retournées par l'algo. liste_scores_d est un param optionnel qui s'il est non vide permet de spécifier que j est l'indice d'une dépêche plutot que d'une image.

In [7]:
def score_sim(score_doc,liste_scores,k):
    res=0
    score_image=liste_scores[k]
    if len(score_image.values())==0 or len(score_doc.values())==0:
        return 0
    max_doc=max(score_doc.values())
    max_im=max(score_image.values())
    for i in score_doc.keys():
        if i in score_image.keys():
            if max_doc==0 or max_im==0:
                return 0
            res += score_doc[i]/max_doc +score_image[i]/max_im
    return res

def best5_image_test(j,liste_scores,liste_scores_d=None):
    rel_descr=range(len(liste_scores))
    
    if liste_scores_d == None:
        doc_score=liste_scores[j]
    else:
        doc_score=liste_scores_d[j]
        
    best_scores=[(i,score_sim(doc_score,liste_scores,i)) for i in rel_descr[:5]]
    best_scores.sort(key = lambda x : x[1])
    for k in rel_descr[5:]:
        score_simil=score_sim(doc_score,liste_scores,k)
        if score_simil > best_scores[0][1]:
            best_scores.pop(0)
            best_scores.append((k,score_sim(doc_score,liste_scores,k)))
            best_scores.sort(key = lambda x : x[1])
    return best_scores

Maintenant on regarde ce que ça donne en normalisant par la somme des scores:

In [10]:
def score_sim_bis(score_doc,liste_scores,k):
    res=0
    score_image=liste_scores[k]
    if len(score_image.values())==0 or len(score_doc.values())==0:
        return 0
    max_doc=max(score_doc.values())
    max_im=max(score_image.values())
    for i in score_doc.keys():
        if i in score_image.keys():
            if max_doc==0 or max_im==0:
                return 0
            res += score_doc[i]/sum(score_doc.values()) +score_image[i]/sum(score_image.values())
    return res

def best5_image_bis_test(j,liste_scores,liste_scores_d=None):
    rel_descr=range(len(liste_scores))
    
    if liste_scores_d == None:
        doc_score=liste_scores[j]
    else:
        doc_score=liste_scores_d[j]
    
    best_scores=[(i,score_sim_bis(doc_score,liste_scores,i)) for i in rel_descr[:5]]
    best_scores.sort(key = lambda x : x[1])
    for k in rel_descr[5:]:
        score_simil=score_sim_bis(doc_score,liste_scores,k)
        if score_simil > best_scores[0][1]:
            best_scores.pop(0)
            best_scores.append((k,score_sim_bis(doc_score,liste_scores,k)))
            best_scores.sort(key = lambda x : x[1])
    return best_scores

## 1er test : on regarde si la meilleure caption selon l'algo est la caption elle meme

In [12]:
df2.reset_index(drop=True,inplace=True)
df_depeche.reset_index(drop=True,inplace=True)

L=[]
for k in range(100):
    l=np.random.randint(0,len(df2))
    while l in L:
        l=np.random.randint(0,len(df2))
    L.append(l)
    
count=[0 for k in range(4)]
for i in tqdm_notebook(L):

    a1=best5_image_test(i,liste_score2)
    a2=best5_image_test(i,liste_score4)
    
    a3=best5_image_bis_test(i,liste_score2)
    
    a4=best5_image_bis_test(i,liste_score4)
    
    for tup in a1:
        if tup[0]==i:
            count[0]+=1
            break
    for tup in a2:
        if tup[0]==i:
            count[1]+=1
            break
    for tup in a3:
        if tup[0]==i:
            count[2]+=1
            break
    for tup in a3:
        if tup[0]==i:
            count[3]+=1
            break
    
count

[80, 65, 91, 91, 0, 0, 0, 0]

On voit que c'est bcp plus performant en normalisant par la somme des scores

## 2eme test : on regarde si dans les 5 photos retournées il  y a l'evenement de la depeche

In [13]:
def ev_match(ev_list1,ev_list2):
    for ev in ev_list1:
        if ev in ev_list2:
            return True
    return False

In [18]:
df2.reset_index(drop=True,inplace=True)
df_depeche.reset_index(drop=True,inplace=True)

L=[]
for k in range(100):
    l=np.random.randint(0,len(df_depeche))
    while l in L:
        l=np.random.randint(0,len(df_depeche))
    L.append(l)
    
count=[0 for k in range(8)]
for i in tqdm_notebook(L):
    a1=best5_image_test(i,liste_score2,liste_score2_d)
    a2=best5_image_test(i,liste_score4,liste_score4_d)
    
    a3=best5_image_bis_test(i,liste_score2,liste_score2_d)
    
    a4=best5_image_bis_test(i,liste_score4,liste_score4_d)
    for tup in a1:
        if ev_match(df_depeche.event[i],df2.event[tup[0]]):
            count[0]+=1
            break
    for tup in a2:
        if ev_match(df_depeche.event[i],df2.event[tup[0]]):
            count[1]+=1
            break
    for tup in a3:
        if ev_match(df_depeche.event[i],df2.event[tup[0]]):
            count[2]+=1
            break
    for tup in a4:
        if ev_match(df_depeche.event[i],df2.event[tup[0]]):
            count[3]+=1
            break

count

[63, 59, 64, 59, 0, 0, 0, 0]